In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import warnings

from collections import defaultdict
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes, svm, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

warnings.filterwarnings('ignore')


In [3]:
''' reading data '''
df = pd.read_csv("C:/Users/Mohan/Documents/Deep Learning R_27.07.21/Deep Learning R_27.07.21/Deep Learning Projects 9 - IMDB Sentiment Analysis using Deep Learning/IMDB Dataset.csv")


In [4]:
''' displaying first 5 rows of data '''
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
''' checking shape of data '''
df.shape


(50000, 2)

In [6]:
''' checking null values '''
df.isnull().sum()


review       0
sentiment    0
dtype: int64

In [7]:
''' checking info of data '''
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [8]:
''' value count of sentiment '''
df.sentiment.value_counts()


sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
''' checking duplicate rows '''
len(df[df.duplicated()])


418

In [10]:
''' dropping duplicate rows '''
df.drop_duplicates(inplace=True)


In [11]:
''' converting every word into lower '''
def lower_word(word):
    return word.lower()

df['review'] = df['review'].map(lower_word)


In [12]:
def regex_(raw_text):
    find_html = re.compile('<.*?>')
    clean_text = re.sub(find_html, '', raw_text)
    return clean_text

''' apply regex_ in review '''
df.review = df.review.apply(lambda x: regex_(x))


In [13]:
df.review


0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 49582, dtype: object

In [14]:
''' Running WhiteSpace tokenizer  '''
w_token = WordPunctTokenizer()
df["review_tokenized"] = [w_token.tokenize(t) for t in df["review"]]


In [15]:
''' displaying first five rows '''
df.head()


,review,sentiment,review_tokenized
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production. the filming tec...,positive,"[a, wonderful, little, production, ., the, fil..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,basically there's a family where a little boy ...,negative,"[basically, there, ', s, a, family, where, a, ..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, ', s, "", love, in, the, time,..."


In [16]:
'''Define POS tags '''
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV


In [17]:
''' Stopwords removal & WordNet lemmatization '''

for idx, t in enumerate(df.review_tokenized):
    if idx % 100 == 0:
        print(idx)

    word_ls = []
    wordnet_lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(t):
        if word not in stopwords.words("english") and word.isalpha():
            word_p = wordnet_lemmatizer.lemmatize(word, tag_map[tag[0]])
            word_ls.append(word_p)
    df.loc[idx, "review_tokenized_cleaned"] = str(word_ls)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [18]:
df.head()


,review,sentiment,review_tokenized,review_tokenized_cleaned
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione...","['one', 'reviewer', 'mention', 'watch', 'oz', ..."
1,a wonderful little production. the filming tec...,positive,"[a, wonderful, little, production, ., the, fil...","['wonderful', 'little', 'production', 'filming..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,...","['think', 'wonderful', 'way', 'spend', 'time',..."
3,basically there's a family where a little boy ...,negative,"[basically, there, ', s, a, family, where, a, ...","['basically', 'family', 'little', 'boy', 'jake..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, ', s, "", love, in, the, time,...","['petter', 'mattei', 'love', 'time', 'money', ..."


In [19]:
''' checking null values '''
df['review_tokenized_cleaned'].isna().sum()


409

In [20]:
''' dropping null values '''
df.dropna(inplace=True)


In [21]:
''' train test split '''
X_train, X_test, y_train, y_test = train_test_split(df['review_tokenized_cleaned'], df['sentiment'], test_size=0.25, 
                                                    random_state=0)


In [22]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)


X_train shape:  (36879,)
X_test shape:  (12294,)


In [23]:
y_train


44934    positive
9456     positive
7708     positive
9458     negative
38835    negative
           ...   
21320    negative
46237    negative
42907    positive
43876    negative
2732     positive
Name: sentiment, Length: 36879, dtype: object

In [24]:
df.sentiment.unique()


array(['positive', 'negative'], dtype=object)

In [25]:
''' LabelEncoding '''
enc = LabelEncoder()
y_train = enc.fit_transform(y_train)
y_test = enc.transform(y_test)


In [26]:
''' TFIDF vector '''
tfidf = TfidfVectorizer(max_features = 5000)

''' fit on data '''
tfidf.fit(df.review_tokenized_cleaned)

''' fit on X_train '''
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)


In [29]:
''' Naive Bayes '''
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

nb = naive_bayes.GaussianNB()

''' fit on data '''
nb.fit(X_train, y_train)


GaussianNB()

In [30]:
''' prediction '''
pred_nb = nb.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, pred_nb))

print("Confusion Matrix: ", confusion_matrix(y_test, pred_nb))


Accuracy Score:  0.5063445583211322
Confusion Matrix:  [[3216 2879]
 [3190 3009]]


In [31]:
print("Classification Report: \n", classification_report(y_test, pred_nb))


Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.53      0.51      6095
           1       0.51      0.49      0.50      6199

    accuracy                           0.51     12294
   macro avg       0.51      0.51      0.51     12294
weighted avg       0.51      0.51      0.51     12294



In [32]:
''' SVM '''
svm_model = svm.SVC(C=1.0, kernel="linear", degree=3, gamma="auto")

''' fit on data '''
svm_model.fit(X_train, y_train)


SVC(gamma='auto', kernel='linear')

In [33]:
''' prediction '''
pred_svm = svm_model.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, pred_svm))

print("Confusion Matrix: ", confusion_matrix(y_test, pred_svm))

print("Classification Report: \n", classification_report(y_test, pred_svm))


Accuracy Score:  0.5015454693346347
Confusion Matrix:  [[2964 3131]
 [2997 3202]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.49      0.49      6095
           1       0.51      0.52      0.51      6199

    accuracy                           0.50     12294
   macro avg       0.50      0.50      0.50     12294
weighted avg       0.50      0.50      0.50     12294



In [34]:
''' Logisctic Regression '''
lg = linear_model.LogisticRegression(solver="lbfgs")

''' fit on data '''
lg.fit(X_train, y_train)


LogisticRegression()

In [35]:
''' prediction '''
pred_lg= svm_model.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, pred_lg))

print("Confusion Matrix: ", confusion_matrix(y_test, pred_lg))

print("Classification Report: \n", classification_report(y_test, pred_lg))


Accuracy Score:  0.5015454693346347
Confusion Matrix:  [[2964 3131]
 [2997 3202]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.49      0.49      6095
           1       0.51      0.52      0.51      6199

    accuracy                           0.50     12294
   macro avg       0.50      0.50      0.50     12294
weighted avg       0.50      0.50      0.50     12294



In [36]:
''' prediction '''
pred_lg= lg.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test, pred_lg))

print("Confusion Matrix: ", confusion_matrix(y_test, pred_lg))

print("Classification Report: \n", classification_report(y_test, pred_lg))


Accuracy Score:  0.5023588742476005
Confusion Matrix:  [[3017 3078]
 [3040 3159]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.49      0.50      6095
           1       0.51      0.51      0.51      6199

    accuracy                           0.50     12294
   macro avg       0.50      0.50      0.50     12294
weighted avg       0.50      0.50      0.50     12294

